## Em busca de novas FEATURES

#### Andre Kunioshi

##### @lumei lin

In [ ]:
from pandas_datareader.data import DataReader
# Stock No
stockNo = 'AAPL'
end = datetime.now()
start = datetime(end.year - 15, end.month, end.day)
df = DataReader(stockNo, 'yahoo', start, end)
print(df.shape)
df.head()

In [ ]:
from statsmodels.tsa.stattools 
import adfuller from numpy 
import log 
df_close = df['Close']
result = adfuller(df_close.dropna()) 

In [ ]:
model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
     test='adf',       # use adftest to find optimal 'd'
     max_p=3, max_q=3, # maximum p and q
     m=1,              # frequency of series
     d=None,           # let model determine 'd'
     seasonal=False,   # No Seasonality
     start_P=0, 
     D=0, 
     trace=True,
     error_action='ignore',  
     suppress_warnings=True, 
     stepwise=True)
print(model_autoARIMA.summary())
model_autoARIMA.plot_diagnostics(figsize=(15,8))
plt.show()

In [ ]:
#Log Transformation
df_log = np.log(df_close)
#split data into train and training set
train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]
# Build Model
ARIMA_model = ARIMA(train_data, order=(0,1,1))  
fitted = ARIMA_model.fit(disp=-1)  
print(fitted.summary())

In [ ]:
# Forecast
fc, se, conf = fitted.forecast(378, alpha=0.05)  # 95% conf
# Make as pandas series
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data.index)
upper_series = pd.Series(conf[:, 1], index=test_data.index)
# Plot
plt.figure(figsize=(10,5), dpi=100)
plt.plot(train_data, label='training data')
plt.plot(test_data, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('%s Stock Price Prediction' %stockNo)
plt.xlabel('Time')
plt.ylabel('%s Stock Price' %stockNo)
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))

##### become-a-pro-in-the-stock-market-in-5-mins

In [ ]:
https://medium.com/@lumeilin301/become-a-pro-in-the-stock-market-in-5-mins-c4bc18c4499f

In [ ]:
# The tech stocks we'll use for this analysis
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']
# Set up End and Start times for data grab
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)
#For loop for grabing yahoo finance data and setting as a dataframe
for stock in tech_list:   
    # Set DataFrame as the Stock Ticker
    stock_info=yf.Ticker(stock)
    hist = stock_info.history(period="max")
    globals()[stock] = hist
company_list = [AAPL, GOOG, MSFT, AMZN]
company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]
for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
#plot the Close values
plt.figure(figsize=(15, 6))
plt.subplots_adjust(top=1.25, bottom=1.2)
for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Close'].plot()
    plt.ylabel('Close')
    plt.xlabel(None)
    plt.title(f"Closing Price of {tech_list[i - 1]}")
    
plt.tight_layout()

In [ ]:
import numpy as np

def log_return(prices):
  return np.log(prices / prices.shift(1))
returns = log_return(AAPL['Close'])

In [ ]:
import plotly.graph_objects as go
AAPL['EMA_9'] = AAPL['Close'].ewm(9).mean().shift()
AAPL['SMA_5'] = AAPL['Close'].rolling(5).mean().shift()
AAPL['SMA_10'] = AAPL['Close'].rolling(10).mean().shift()
AAPL['SMA_15'] = AAPL['Close'].rolling(15).mean().shift()
AAPL['SMA_30'] = AAPL['Close'].rolling(30).mean().shift()
fig = go.Figure()
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.SMA_5, name='SMA 5'))
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.SMA_10, name='SMA 10'))
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.SMA_15, name='SMA 15'))
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.SMA_30, name='SMA 30'))
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.Close, name='Close', opacity=0.2))
fig.show()

In [ ]:
Relative Strength Index(RSI). It’s used to predict whether a stock is overbought/oversold

In [ ]:
def relative_strength_idx(df, n=14):
    close = df['Close']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi
AAPL['RSI'] = relative_strength_idx(hist).fillna(0)
fig = go.Figure(go.Scatter(x=AAPL.Date, y=AAPL.RSI, name='RSI'))
fig.show()

In [ ]:
EMA_12 = pd.Series(AAPL['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(AAPL['Close'].ewm(span=26, min_periods=26).mean())
AAPL['MACD'] = pd.Series(EMA_12 - EMA_26)
AAPL['MACD_signal'] = pd.Series(AAPL.MACD.ewm(span=9, min_periods=9).mean())
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL.Close, name='Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=AAPL.Date, y=EMA_12, name='EMA 12'), row=1, col=1)
fig.add_trace(go.Scatter(x=AAPL.Date, y=EMA_26, name='EMA 26'), row=1, col=1)
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL['MACD'], name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=AAPL.Date, y=AAPL['MACD_signal'], name='Signal line'), row=2, col=1)
fig.show()

In [ ]:
AAPL.var() 
#628.3749178125261

In [ ]:
AAPL.cov()

In [ ]:
Find the most important technical index

In [ ]:
#prepare for the train/test dataset
test_size  = 0.15
valid_size = 0.15
test_split_idx  = int(df.shape[0] * (1-test_size))
valid_split_idx = int(df.shape[0] * (1-(valid_size+test_size)))
train_df  = df.loc[:valid_split_idx].copy()
valid_df  = df.loc[valid_split_idx+1:test_split_idx].copy()
test_df   = df.loc[test_split_idx+1:].copy()
drop_cols = ['Date']
train_df = train_df.drop(drop_cols, 1)
valid_df = valid_df.drop(drop_cols, 1)
test_df  = test_df.drop(drop_cols, 1)
y_train = train_df['Close'].copy()
X_train = train_df.drop(['Close'], 1)
y_valid = valid_df['Close'].copy()
X_valid = valid_df.drop(['Close'], 1)
y_test  = test_df['Close'].copy()
X_test  = test_df.drop(['Close'], 1)

#Fine-tune XGBoostRegressor
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}
eval_set = [(X_train, y_train), (X_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)
model = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)
#plot the feature importance
plot_importance(model);
importances = model.feature_importances_
print(importances)